### INSTALACION DE LOS PAQUETES NECESARIOS

In [6]:
! pip install -U spacy
! python -m spacy download en
! pip install spacy
! pip install spacymoji
! pip install emoji
! pip install scikit-learn
! pip install gensim
! pip install sklearn


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.0/en_core_web_sm-3.7.0-py3-none-any.whl (12.8 MB)
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import spacy
import emoji
import numpy as np
from tqdm import tqdm

from sklearn.cluster import DBSCAN
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.test.utils import get_tmpfile

# 1. Preparación del Dataset

## 1.1. CARGAR FICHERO CSV CON LOS DATOS

In [8]:
datsetPath = '../Datasets/corto.csv'
#datsetPath = '../Datasets/SuicideDetection.csv'

data = pd.read_csv(datsetPath)

data.info()
data.head(n=5)

FileNotFoundError: [Errno 2] No such file or directory: '../Datasets/Suicide_Detection.csv'

In [ ]:
from matplotlib import pyplot as plt

barPlot = plt.bar(data['class'].value_counts().index, data['class'].value_counts())
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.bar_label(barPlot)
plt.show()

## 1.2. SELECCIONAR LA COLUMNA DE TEXTOS (DISCRIMINAR LA CLASE)

In [4]:
textos = []
for instancia in data.values:
    textos.append(instancia[1])

# 2. Pre-proceso
## 2.1. LIMPIEZA Y FORMARTEO DE LOS DATOS

In [5]:
# Cuidado! La ejecución de este bloque con 232074 instancias, toma aproximademente 5 min? 

nlp = spacy.load("en_core_web_sm") # Cargar modelo
nlp.add_pipe("emoji", first=True)

textos_tokenizados = []
for texto in tqdm(textos, desc="Procesando textos"):
    texto = emoji.demojize(texto)  # Emojis a texto
    doc = nlp(texto)
    lexical_tokens = [token.lemma_.lower() for token in doc if len(token.text) > 3 and token.is_alpha]
    textos_tokenizados.append(lexical_tokens)

Procesando textos:   0%|          | 596/232074 [00:58<6:18:33, 10.19it/s] 


KeyboardInterrupt: 

In [ ]:
print(textos[0])
print(textos_tokenizados[0])

## 2.2. Espacio de decisión - VECTORIZACION DE LOS DATOS (DOC2VEC)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(textos_tokenizados)]
model = Doc2Vec(documents, vector_size=150, window=2, dm=1, epochs=100, workers=4)
# DM siempre a 1 para que sea distributed memory (que tenga en cuenta el contexto)
# Window se puede cambiar pero tener en cuenta que cuanto mayor sea mas coste genera
# Negative: siempre a 0 para que aprenda que palabras son ruido
#Los que se pueden probar para parametro ekorketa:
    # VECTOR SIZE!!!! (tamaño del embedding: dimensiones)
    # Epochs: numero de iteraciones por el dataset
    # Alpha: learning rate inicial
    # Min alpha: El valor de LR con el que va acabar → Igual no poner?
    # Max_vocab_size: Limita el tamaño maximo de las palabras a tener en cuenta (borra las menos frecuentes)

model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

model.save(get_tmpfile("my_doc2vec_model"))

#vector = model.infer_vector(['wife', 'threaten', 'leave', 'wife', 'good', 'because', 'cheat', 'twice', 'lie', 'much', 'that', 'have', 'decide', 'refuse', 'back', 'day', 'begin', 'threaten', 'suicide', 'have', 'tirelessly', 'spend', 'these', 'paat', 'day', 'talk', 'keep'])

#similar_documents = model.dv.most_similar(vector) # Deberia salir 0 porque es el documento del que se han sacado todas las palabras

# 3. Clustering

## 3.1. Aplicando algoritmo DBSCAN (PREDETERMINADO) de la libería sklearn

In [ ]:
document_vectors = [model.infer_vector(doc) for doc in textos_tokenizados]

# Aplicar DBSCAN a los vectores de documentos
dbscan = DBSCAN(eps=2, min_samples=2, leaf_size=5)  # Ajusta los parámetros según tu caso
labels = dbscan.fit_predict(np.array(document_vectors))

# Los resultados del clustering están en 'labels'
print("Etiquetas de clusters:", labels)